In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
# import wandb


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [10, 20, 30]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
    },
}


# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")


train_df = pd.read_csv('train.csv')
train_df.head()
train_df['stars'] = train_df['stars'] - 1



import math
SAVE_EVERY_N_EPOCHS = 15
steps_per_epoch = math.floor(len(train_df) / SAVE_EVERY_N_EPOCHS)
if(len(train_df) % SAVE_EVERY_N_EPOCHS > 0):
    steps_per_epoch +=1

eval_df = pd.read_csv('test.csv')

# Optional model configuration
# model_args = ClassificationArgs(num_train_epochs=100, use_multiprocessing=True,
#                                 overwrite_output_dir=True,
#                                 train_batch_size=16,
#                                 save_model_every_epoch=False,
#                                 reprocess_input_data=True,
#                                 save_steps= steps_per_epoch * SAVE_EVERY_N_EPOCHS)


model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = False
model_args.manual_seed = 31
model_args.use_multiprocessing = True
model_args.train_batch_size = 32
model_args.learning_rate = 1e-5
# model_args.eval_batch_size = 8
# model_args.wandb_project = "Simple Sweep"
model_args.num_train_epochs=25
model_args.save_steps = steps_per_epoch * SAVE_EVERY_N_EPOCHS
model_args.save_model_every_epoch = False

# wandb.init()
# Create a ClassificationModel
model = ClassificationModel(
    "bert",
    "bert-base-cased",
    num_labels=5, args=model_args, use_cuda = True,
    # sweep_config=wandb.config
)

# Train the model
model.train_model(train_df)
# wandb.join()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

Running Epoch 22 of 25:   0%|          | 0/188 [00:00<?, ?it/s]

Running Epoch 23 of 25:   0%|          | 0/188 [00:00<?, ?it/s]

Running Epoch 24 of 25:   0%|          | 0/188 [00:00<?, ?it/s]

Running Epoch 25 of 25:   0%|          | 0/188 [00:00<?, ?it/s]

(4700, 0.2194176708748366)

In [ ]:
# Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
text_df = pd.DataFrame(eval_df['text'])
# model.predict(text_df)
stars, _ = model.predict(eval_df['text'].tolist())
# text_df

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
import numpy as np

n_df = pd.DataFrame({
    'ID': eval_df['ID'],
    'starts': np.array(stars) + 1
})

n_df.to_csv('sub-DC2.csv', index=False)